In [1]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [6]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

In [7]:
messages = chat_template.invoke({"name": "Bob", "user_input": "What is your name?"})
messages

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')])

## ChatPromptTemplate + core messages

In [8]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

In [9]:

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            (
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things")]


# Asistente legal prompts

In [29]:
from hydra.utils import instantiate
from omegaconf import OmegaConf
from llm_agent.utils.paths import PROMPTS_CONFIG_PATH

In [35]:
with (PROMPTS_CONFIG_PATH / "system.yaml") as fp:
    system_config = OmegaConf.load(fp)
    OmegaConf.resolve(system_config)
    system_config = instantiate(system_config)
    
with (PROMPTS_CONFIG_PATH / "human.yaml") as fp:
    human_config = OmegaConf.load(fp)
    OmegaConf.resolve(human_config)
    human_config = instantiate(human_config)

In [43]:
system_prompt = SystemMessagePromptTemplate.from_template((system_config.agent_base_system_prompt))
system_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ccaa', 'country'], template='Eres un asistente legal que debe seguir la siguientes instrucciones:\nNombre del Asistente: Asistente Legal Ámbito de Operación: Comunidad Autónoma {ccaa}, {country} Idioma de Operación: Español\nFunciones Principales: 1. Asistir en todo tipo de tareas legales específicas de la Comunidad Autónoma [Nombre de la Comunidad]. 2. Proporcionar apoyo y asesoramiento confiable y preciso en cuestiones jurídicas. 3. Mantener las interacciones y respuestas exclusivamente en español. 4. Limitar las actividades y respuestas a temas estrictamente relacionados con el derecho.\nInstrucciones Adicionales: - El asistente debe priorizar la precisión y la actualidad en la información legal proporcionada. - Se debe garantizar la confidencialidad y la seguridad de los datos en todas las interacciones. - El asistente debe ser capaz de referir al usuario a recursos legales adicionales o expertos en derecho cuando 

In [45]:
human_prompt_1 = HumanMessagePromptTemplate.from_template(human_config.generate_document_prompt)
human_prompt_1

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'message'], template='Debes ayudar en la siguiente tarea: {message}.   Utiliza la siguiente información para realizar la taera: {context}.\n'))

In [46]:
human_prompt_2 = HumanMessagePromptTemplate.from_template(human_config.generate_instance_prompt)
human_prompt_2

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Debes escribir una instancia a partir de la informacion proporcionada. Esta debe seguir la siguiente estructura: presentacion, exposición, solicitud, fecha, lugar y firma. Escribelo usando la siguiente información: {context}'))

In [47]:
chat_template = ChatPromptTemplate.from_messages(
    [
        system_prompt,
        human_prompt_1,
    ]
)

In [53]:
print(chat_template.invoke({"ccaa": "Cataluña", "country": "España", "message": "Contrato de arrendamiento", "context": "DNI 12345678V"}).to_string())

System: Eres un asistente legal que debe seguir la siguientes instrucciones:
Nombre del Asistente: Asistente Legal Ámbito de Operación: Comunidad Autónoma Cataluña, España Idioma de Operación: Español
Funciones Principales: 1. Asistir en todo tipo de tareas legales específicas de la Comunidad Autónoma [Nombre de la Comunidad]. 2. Proporcionar apoyo y asesoramiento confiable y preciso en cuestiones jurídicas. 3. Mantener las interacciones y respuestas exclusivamente en español. 4. Limitar las actividades y respuestas a temas estrictamente relacionados con el derecho.
Instrucciones Adicionales: - El asistente debe priorizar la precisión y la actualidad en la información legal proporcionada. - Se debe garantizar la confidencialidad y la seguridad de los datos en todas las interacciones. - El asistente debe ser capaz de referir al usuario a recursos legales adicionales o expertos en derecho cuando sea necesario.

Human: Debes ayudar en la siguiente tarea: Contrato de arrendamiento.   Utili